# Homework 3: Building an NDArray library

In this homework, you will build a simple backing library for the processing that underlies most deep learning systems: the n-dimensional array (a.k.a. the NDArray).  Up until now, you have largely been using numpy for this purpose, but this homework will walk you through developing what amounts to your own (albeit much more limited) variant of numpy, which will support both CPU and GPU backends.  What's more, unlike numpy (and even variants like PyTorch), you won't simply call out to existing highly-optimized variants of matrix multiplication or other manipulation code, but actually write your own versions that are reasonably competitive will the highly optimized code backing these standard libraries (by some measure, i.e., "only 2-3x slower" ... which is a whole lot better than naive code that can easily be 100x slower).  This class will ultimately be integrated into `needle`, but for this assignment you can _only_ focus on the ndarray module, as this will be the only subject of the tests.

在这个作业中，你将构建一个简单的支持深度学习系统基础处理的后端库：n维数组（也称为NDArray）。 目前为止，你在很大程度上一直在使用numpy来实现这个目的，但这个作业将带你逐步开发自己的（尽管功能更有限）numpy变体，它将支持CPU和GPU后端。 更重要的是，与numpy（甚至像PyTorch这样的变体）不同，你不会简单地调用现有高度优化的矩阵乘法或其他操作代码，而是实际编写自己的版本，这些版本在某种程度上与支持这些标准库的高度优化代码竞争力相当（以某种度量，“仅比快速优化代码慢2-3倍”...这比容易慢100倍的朴素代码好多了）。 这个类最终将集成到`needle`中，但在本次作业中，你只能专注于ndarray模块，因为这将是测试的唯一主题。

In [ ]:
# Code to set up the assignment
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/
# !mkdir -p 10714
# %cd /content/drive/MyDrive/10714
# !git clone https://github.com/dlsys10714/hw3.git
# %cd /content/drive/MyDrive/10714/hw3

!pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
!pip3 install pybind11

In [38]:
!make clean
!make

rm -rf build python/needle/backend_ndarray/ndarray_backend*.so
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PythonInterp: /home/haozhe/anaconda3/bin/python (found suitable version "3.11.5", minimum required is "3.6") 
-- Found PythonLibs: /home/haozhe/anaconda3/lib/libpython3.11.so
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- Found pybind11: /home/haozhe/anaconda3/lib/python3.11/site-packages/pybind11/include (found version "2.11.1")
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performi

In [39]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


In [40]:
import sys
sys.path.append('./python')

## Getting familiar with the NDArray class

As you get started with this homework, you should first familiarize yourself with the `NDArray.py` class we have provided as a starting point for the assignment.  The code is fairly brief (it's ~500 lines, but a lot of these are comments provided for the functions you'll implement).

At its core, the NDArray class is a Python wrapper for handling operations on generic n-dimensional arrays.  Recall that virtually any such array will be stored internally as a vector of floating point values, i.e.,

```c++
float data[size];
```

and then the actual access to different dimensions of the array are all handled by additional fields (such as the array shape, strides, etc) that indicates how this "flat" array maps to n-dimensional structure.  In order to achieve any sort of reasonable speed, the "raw" operations (like adding, binary operations, but also more structured operations like matrix multiplication, etc), all need to be written at some level in some native language like C++ (including e.g., making CUDA calls).  But a large number of operations likes transposing, broadcasting, sub-setting of matrices, and other, can all be handled by just adjusting the high-level structure of the array, like it's strides.

在开始这项作业时，你应该先熟悉我们提供的`NDArray.py`类，作为这个任务的起点。这段代码相当简洁（大约500行，但很多是针对你将实现的函数提供的注释）。

在其核心，NDArray类是一个用于处理通用n维数组操作的Python包装器。请记住，几乎任何这样的数组都会在内部以浮点值向量的形式存储，即

```c++
float data[size];
```

然后，对数组的不同维度的实际访问都由额外的字段（如数组形状、步幅等）处理，这些字段指示这个“扁平”数组如何映射到n维结构。为了实现任何合理的速度，所有“原始”操作（如加法、二进制操作，以及更结构化的操作，如矩阵乘法等）都需要在某种本地语言（如C++）的某个级别编写（包括例如进行CUDA调用）。但是，许多操作，如转置、广播、子集取值等，都可以通过调整数组的高级结构，比如步幅，来处理。

---

The philosophy behind the NDArray class is that we want _all_ the logic for handling this structure of the array to be written in Python.  Only the "true" low level code that actually performs the raw underlying operations on the flat vector (as well as the code to manage these flat vectors, as they might need to e.g., be allocated on GPUs), is written in C++.  The precise nature of this separation will likely start to make more sense to you as you work through the assignment, but generally speaking everything that can be done in Python, is done in Python; often e.g., at the cost of some inefficiencies ... we call `.compact()` (which copies memory) liberally in order to make the underlying C++ implementations simpler.

In more detail, there are five fields within the NDArray class that you'll need to be familiar with (note that the real class member these all these fields is preceded by an underscore, e.g., `_handle`, `_strides`, etc, some of which are then exposed as a public property ... for all your code it's fine to use the internal, underscored version).

1. `device` - A object of type `BackendDevice`, which is a simple wrapper that contains a link to the underlying device backend (e.g., CPU or CUDA).
2. `handle` - A class objected that stores the underlying memory of the array.  This is allocated as a class of type `device.Array()`, though this allocation all happens in the provided code (specifically the `NDArray.make` function), and you don't need to worry about calling it yourself.
3. `shape` - A tuple specifying the size of each dimension in the array.
4. `strides` - A tuple specifying the strides of each dimension in the array.
5. `offset` - An integer indicating where in the underlying `device.Array` memory the array actually starts (it's convenient to store this so we can more easily manage pointing back to existing memory, without having to track allocations).

NDArray类背后的理念是，我们希望所有处理数组结构的逻辑都用Python编写。只有实际执行原始底层操作的“真正”低级代码（以及管理这些平坦向量的代码，因为它们可能需要在GPU上分配），才用C++编写。这种分离的确切性质可能在您完成作业时开始变得更加清晰，但一般来说，所有可以在Python中完成的事情都会在Python中完成；通常会牺牲一些效率...我们经常自由地调用`.compact()`（复制内存），以使底层C++实现更简单。

更详细地说，您需要熟悉NDArray类中的五个字段（请注意，所有这些字段的真实类成员都以下划线开头，例如，`_handle`，`_strides`等等，其中一些然后作为公共属性公开...对于您的所有代码来说，使用内部的下划线版本是可以的）。

1. `device` - 一个类型为 `BackendDevice` 的对象，它是一个简单的包装器，包含了指向底层设备后端（例如 CPU 或 CUDA）的链接。
2. `handle` - 一个类对象，用于存储数组的底层内存。这被分配为 `device.Array()` 类型的类，尽管这个分配都是在提供的代码中进行的（特别是 `NDArray.make` 函数），你不需要担心自己调用它。
3. `shape` - 一个元组，指定数组中每个维度的大小。
4. `strides` - 一个元组，指定数组中每个维度的步幅。
5. `offset` - 一个整数，指示数组实际开始的地方在底层的 `device.Array` 内存中（将其存储起来很方便，这样我们可以更容易地管理指向现有内存的指针，而不必跟踪分配）。

---

By manipulating these fields, even pure Python code can perform a lot of the needed operations on the array, such as permuting the dimensions (i.e., transposing), broadcasting, and more.  And then for the raw array manipulation calls, the `device` class has a number of methods (implemented in C++) that contains the necessary implementations.

There are a few points to note:

* Internally, the class can use _any_ efficient means of operating on arrays of data as a "device" backend.  Even, for example, a numpy array, but where instead of actually using the `numpy.ndarray` to represent the n-dimensional array, we just represent a "flat" 1D array in numpy, then call the relevant numpy methods to implement all the needed operators on this raw memory.  This is precisely what we do in the `ndarray_backend_numpy.py` file, which essentially provided a "stub reference" that just uses numpy for everything.  You can use this class to help you  better debug your own "real" implementations for the "native" CPU and GPU backends.
* Of particular importance for many of your Python implementations will be the `NDArray.make` call:
```python
def make(shape, strides=None, device=None, handle=None, offset=0):
```
which creates a new NDArray with the given shape, strides, device, handle, and offset.  If `handle` is not specified (i.e., no pre-existing memory is referenced), then the call will allocate the needed memory, but if handle _is_ specified then no new memory is allocated, but the new NDArray points the same memory as the old one.  It is important to efficient implementations that as many of your functions as possible _don't_ allocate new memory, so you will want to use this call in many cases to accomplish this.
* The NDArray has a `.numpy()` call that converts the array to numpy.  This is _not_ the same as the "numpy_device" backend: this creates an actual `numpy.ndarray` that is equivalent to the given NDArray, i.e., the same dimensions, shape, etc, though not necessarily the same strides (Pybind11 will reallocate memory for matrices that are returned in this manner, which can change the striding).


通过操纵这些字段，即使是纯Python代码也可以在数组上执行许多所需的操作，比如置换维度（即转置）、广播等。然后对于原始数组操作调用，`device`类有许多方法（在C++中实现），其中包含了必要的实现。

有几点需要注意：

* 在内部，该类可以使用_任何_高效操作数据数组的方式作为“设备”后端。甚至，例如，一个numpy数组，但是我们不是实际使用`numpy.ndarray`来表示n维数组，而是在numpy中表示一个“平坦”的1D数组，然后调用相关的numpy方法来实现对这个原始内存的所有所需操作符。这正是我们在`ndarray_backend_numpy.py`文件中所做的，它基本上提供了一个仅使用numpy的“存根引用”。您可以使用此类来帮助您更好地调试自己的“真实”实现，用于“本机”CPU和GPU后端。
* 对于您的许多Python实现特别重要的是`NDArray.make`调用：

```python
def make(shape, strides=None, device=None, handle=None, offset=0):
```

它使用给定的形状、步幅、设备、句柄和偏移量创建一个新的NDArray。如果未指定`handle`（即没有引用预先存在的内存），那么调用将分配所需的内存，但如果指定了`handle`，则不会分配新的内存，而是新的NDArray指向与旧的相同的内存。对于尽可能多的函数不分配新内存是非常重要的，因此在许多情况下您将希望使用此调用来实现这一点。

* NDArray有一个`.numpy()`调用，将数组转换为numpy。这与“numpy_device”后端不同：这将创建一个实际的`numpy.ndarray`，它等价于给定的NDArray，即相同的维度、形状等，尽管不一定是相同的步幅（Pybind11将为以这种方式返回的矩阵重新分配内存，这可能会改变步幅）。

## Part 1: Python array operations

As a starting point for your class, implement the following functions in the `ndarray.py` file:

- `reshape()`
- `permute()`
- `broadcast_to()`
- `__getitem__()`

The inputs/outputs of these functions are all described in the docstring of the function stub.  It's important to emphasize that _none_ of these functions should reallocate memory, but should instead return NDArrays that share the same memory with `self`, and just use clever manipulation of shape/strides/etc in order to obtain the necessary transformations.

One thing to note is that the `__getitem__()` call, unlike numpy, will never change the number of dimensions in the array.  So e.g., for a 2D NDArray `A`, `A[2,2]` would return a still-2D with one row and one column.  And e.g. `A[:4,2]` would return a 2D NDarray with 4 rows and 1 column.

You can rely on the `ndarray_backend_numpy.py` module for all the code in this section.  You can also look at the results of equivalent numpy operations (the test cases should illustrate what these are).

After implementing these functions, you should pass/submit the following tests.  Note that we test all of these four functions within the test below, and you can incrementally try to pass more tests as you implement each additional function.

作为课程的起点，在`ndarray.py`文件中实现以下函数：

- `reshape()`
- `permute()`
- `broadcast_to()`
- `__getitem__()`

这些函数的输入/输出都在函数存根的文档字符串中有描述。重要的是要强调，_所有_这些函数都不应重新分配内存，而应该返回与`self`共享相同内存的NDArrays，并且仅使用形状/步幅等的巧妙操作来获得所需的转换。

需要注意的一点是，与numpy不同，`__getitem__()`调用永远不会改变数组的维数。例如，对于一个2D NDArray `A`，`A[2,2]`会返回一个仍然是2D的数组，具有一行和一列。例如，`A[:4,2]`会返回一个具有4行和1列的2D NDArray。

您可以依赖`ndarray_backend_numpy.py`模块中的所有代码来完成本节中的工作。您也可以查看等效numpy操作的结果（测试案例应说明这些操作是什么）。

在实现这些函数之后，您应该通过/提交以下测试。请注意，我们在下面的测试中测试所有这四个函数，您可以在实现每个额外函数时逐步尝试通过更多测试。

---

In [41]:
!python3 -m pytest -v -k "(permute or reshape or broadcast or getitem) and cpu and not compact" -s

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collecting ... Using needle backend
collected 138 items / 127 deselected / 11 selected                             

tests/hw3/test_ndarray.py::test_permute[cpu-params0] PASSED
tests/hw3/test_ndarray.py::test_permute[cpu-params1] PASSED
tests/hw3/test_ndarray.py::test_permute[cpu-params2] PASSED
tests/hw3/test_ndarray.py::test_reshape[cpu-params0] PASSED
tests/hw3/test_ndarray.py::test_reshape[cpu-params1] PASSED
tests/hw3/test_ndarray.py::test_getitem[cpu-params0] PASSED
tests/hw3/test_ndarray.py::test_getitem[cpu-params1] PASSED
tests/hw3/test_ndarray.py::test_getitem[cpu-params2] PASSED
tests/hw3/test_ndarray.py::test_getitem[cpu-params3] PASSED
tests/hw3/test_ndarray.py::test_getitem[cpu-params4] PASSED
tests/hw3/test_nda

In [42]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_python_ops"

## Part 2: CPU Backend - Compact and setitem

Implement the following functions in `ndarray_backend_cpu.cc`:
* `Compact()`
* `EwiseSetitem()`
* `ScalarSetitem()`

To see why these are all in the same category, let's consider the implementation of the `Compact()` function.  Recall that a matrix is considered compact if it is layed out sequentially in memory in "row-major" form (but really a generalization of row-many to higher dimensional arrays), i.e. with the last dimension first, followed by the second to last dimension, etc, all the way to the first.  In our implementation, we also require that the total size of allocated backend array be equal to the size of the array (i.e., the underlying array also can't have any data before or after the array data, which e.g., implies that the `.offset` field equals zero).

在`ndarray_backend_cpu.cc`中实现以下功能：
* `Compact()`
* `EwiseSetitem()`
* `ScalarSetitem()`

要了解为什么它们都属于同一类别，让我们考虑`Compact()`函数的实现。回想一下，如果矩阵在内存中以“行优先”形式依次布局（实际上是对高维数组的一般化），则矩阵被视为紧凑。也就是说，最后一个维度首先，然后是倒数第二个维度，依此类推，一直到第一个维度。在我们的实现中，我们还要求分配的后端数组的总大小等于数组的大小（即，底层数组也不能在数组数据之前或之后有任何数据，这意味着`.offset`字段等于零）。

---

Now let's consider, using a 3D array as a an example, of how a compact call might work.  Here `shape` and `strides` are the shape and strides of the matrix being compacted (i.e., before we have compacted it).

```c++
cnt = 0;
for (size_t i = 0; i < shape[0]; i++)
    for (size_t j = 0; j < shape[1]; j++)
        for (size_t k = 0; k < shape[2]; k++)
            out[cnt++] = in[strides[0]*i + strides[1]*j + strides[2]*k];
```
In other words, we're converting from a stride-based representation of the matrix to a purely sequential one.

现在让我们来考虑，以一个3D数组为例，看看紧凑调用是如何工作的。这里的`shape`和`strides`是被紧缩的矩阵的形状和步幅（即在我们对其进行紧缩之前）。

```c++
...code省略复制
```

换句话说，我们正在将矩阵从基于步幅的表示转换为纯粹的顺序表示。

---

Now, the challenge in implementing `Compact()` is that you want the method to work for any number of input dimensions.  It's easy to specialize for different fixed-dimension-size arrays, but for a generic implementation, you'll want to think about how to do this same operation where you effectively want a "variable number of for loops".  As a hint, one way to do this is to maintain a vector of indices (of size equal to the number of dimensions), and then manually increment them in a loop (including a "carry" operation when any of the reaches their maximum size).

However, if you get really stuck with this, you can alway use the fact that we're probably not going to ask you to deal with matrices of more than 6 dimensions (though we _will_ use 6 dimensions, for the im2col operation we discussed in class).

现在，在实现`Compact()`中的挑战在于你希望该方法适用于任意数量的输入维度。对于不同固定维度大小的数组，专门化很容易，但对于通用实现，你需要考虑如何执行相同的操作，实际上你需要一个“可变数量的for循环”。作为提示，一种方法是维护一个索引向量（大小等于维度数量），然后在循环中手动递增它们（包括在任何达到其最大大小时进行“进位”操作）。

然而，如果你在这方面遇到真正困难，你可以利用这样一个事实，即我们可能不会要求你处理超过6维的矩阵（尽管我们将使用6维，用于我们在课堂上讨论的im2col操作）。

---

#### The connection to setitem
The setitem functionality, while seemingly quite different, is actually intimately related to `Compact()`.  `__setitem()__` is the Python function called when setting some elements of the object, i.e.,
```python
A[::2,4:5,9] = 0 # or = some_other_array
```
How would you go about implementing this?  In the `__getitem()__` call above, you already implemented a method to take a subset of a matrix without copying (but just modifying strides).  But how would you actually go about _setting_ elements of this array?  In virtually all the other settings in this homework, we call `.compact()` before setting items in an output array, but in this case it doesn't work: calling `.compact()` would copy the subset array to some new memory, but the whole point of the `__setitem__()` call is that we want to modify existing memory.

If you think about this for a while, you'll realize that the answer looks a lot like `.compact()` but in reverse.  If we want to assign a (itself already compact) right hand side matrix to a `__getitem()__` results, then we need to here like `shape` and `stride` be those fields of the _output_ matrix.  Then we could implement the setitem call as follows

虽然`setitem`功能看起来有些不同，但实际上与`Compact()`密切相关。`__setitem()__`是在设置对象的某些元素时调用的Python函数，即，

```python
A[::2,4:5,9] = 0 # 或 = some_other_array
```

你会如何实现这个？在上面的`__getitem()__`调用中，你已经实现了一个方法来获取矩阵的子集而不是复制（只是修改步幅）。但是你实际上会如何去_设置_这个数组的元素？在本次作业的几乎所有其他设置中，我们在设置输出数组中的项之前调用`.compact()`，但在这种情况下不起作用：调用`.compact()`将复制子集数组到一些新的内存，但`__setitem__()`调用的整个重点是我们想要修改现有的内存。

如果你仔细考虑一下，你会意识到答案看起来很像`.compact()`但是相反。如果我们想要将一个（本身已经被压缩的）右侧矩阵赋给`__getitem()__`的结果，那么我们需要在这里像`形状`和`步幅`这样的字段是输出矩阵的。然后我们可以实现setitem调用如下：

---

```c++
cnt = 0;
for (size_t i = 0; i < shape[0]; i++)
    for (size_t j = 0; j < shape[1]; j++)
        for (size_t k = 0; k < shape[2]; k++)
            out[strides[0]*i + strides[1]*j + strides[2]*k] = in[cnt++]; // or "= val;"
```
Due to this similarity, if you implement your indexing strategy in a modular fashion, you'll be able to reuse it between the `Compact()` call and the `EwiseSetitem()` and `ScalarSetitem()` calls.

由于这种相似性，如果你以模块化的方式实施你的索引策略，你就能在`Compact()`调用以及`EwiseSetitem()`和`ScalarSetitem()`调用之间重复使用它。

In [43]:
!python3 -m pytest -v -k "(compact or setitem) and cpu"

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collected 138 items / 123 deselected / 15 selected                             

tests/hw3/test_ndarray.py::test_compact[cpu-transpose] PASSED            [  6%]
tests/hw3/test_ndarray.py::test_compact[cpu-broadcast_to] PASSED         [ 13%]
tests/hw3/test_ndarray.py::test_compact[cpu-reshape1] PASSED             [ 20%]
tests/hw3/test_ndarray.py::test_compact[cpu-reshape2] PASSED             [ 26%]
tests/hw3/test_ndarray.py::test_compact[cpu-reshape3] PASSED             [ 33%]
tests/hw3/test_ndarray.py::test_compact[cpu-getitem1] PASSED             [ 40%]
tests/hw3/test_ndarray.py::test_compact[cpu-getitem2] PASSED             [ 46%]
tests/hw3/test_ndarray.py::test_compact[cpu-transposegetitem] PASSED     [ 53%]
tests/hw3/test

In [44]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cpu_compact_setitem"

## Part 3: CPU Backend - Elementwise and scalar operations

Implement the following functions in `ndarray_backend_cpu.cc`:

* `EwiseMul()`, `ScalarMul()`
* `EwiseDiv()`, `ScalarDiv()`
* `ScalarPower()`
* `EwiseMaximum()`, `ScalarMaximum()`
* `EwiseEq()`, `ScalarEq()`
* `EwiseGe()`, `ScalarGe()`
* `EwiseLog()`
* `EwiseExp()`
* `EwiseTanh()`

You can look at the included
`EwiseAdd()` and `ScalarAdd()` functions (plus the invocations from `NDArray` in order to understand the required format of these functions.

Note that unlike the remaining functions mentioned here, we do not include function stubs for each of these functions.  This is because, while you can implement these naively just through implementing each function separately, though this will end up with a lot of duplicated code.  You're welcome to use e.g., C++ templates or macros to address this problem (but these would only be exposed internally, not to the external interface).

**Note**: Remember to register functions  in the pybind module after finishing your implementations.

In [45]:
!python3 -m pytest -v -k "(ewise_fn or ewise_max or log or exp or tanh or (scalar and not setitem)) and cpu"

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collected 138 items / 115 deselected / 23 selected                             

tests/hw3/test_ndarray.py::test_ewise_fn[cpu-shape0-multiply] PASSED     [  4%]
tests/hw3/test_ndarray.py::test_ewise_fn[cpu-shape0-divide] PASSED       [  8%]
tests/hw3/test_ndarray.py::test_ewise_fn[cpu-shape0-add] PASSED          [ 13%]
tests/hw3/test_ndarray.py::test_ewise_fn[cpu-shape0-subtract] PASSED     [ 17%]
tests/hw3/test_ndarray.py::test_ewise_fn[cpu-shape0-equal] PASSED        [ 21%]
tests/hw3/test_ndarray.py::test_ewise_fn[cpu-shape0-greater_than] PASSED [ 26%]
tests/hw3/test_ndarray.py::test_ewise_fn[cpu-shape1-multiply] PASSED     [ 30%]
tests/hw3/test_ndarray.py::test_ewise_fn[cpu-shape1-divide] PASSED       [ 34%]
tests/hw3/test

In [46]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cpu_ops"

## Part 4: CPU Backend - Reductions


Implement the following functions in `ndarray_backend_cpu.cc`:

* `ReduceMax()`
* `ReduceSum()`

In general, the reduction functions `.max()` and `.sum()` in NDArray take the max or sum across a specified axis specified by the `axis` argument (or across the entire array when `axis=None`); note that we don't support axis being a set of axes, though this wouldn't be too hard to add if you desired (but it's not in the interface you should implement for the homework).

Because summing over individual axes can be a bit tricky, even for compact arrays, these functions (in Python) in Python simplify things by permuting the last axis to the be the one reduced over (this is what the `reduce_view_out()` function in NDArray does), then compacting the array.  So for your `ReduceMax()` and `ReduceSum()` functions you implement in C++, you can assume that both the input and output arrays are contiguous in memory, and you want to just reduce over contiguous elements of size `reduce_size` as passed to the C++ functions. 

In [47]:
!python3 -m pytest -v -k "reduce and cpu"

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collected 138 items / 130 deselected / 8 selected                              

tests/hw3/test_ndarray.py::test_reduce_sum[params0-cpu] PASSED           [ 12%]
tests/hw3/test_ndarray.py::test_reduce_sum[params1-cpu] PASSED           [ 25%]
tests/hw3/test_ndarray.py::test_reduce_sum[params2-cpu] PASSED           [ 37%]
tests/hw3/test_ndarray.py::test_reduce_sum[params3-cpu] PASSED           [ 50%]
tests/hw3/test_ndarray.py::test_reduce_max[params0-cpu] PASSED           [ 62%]
tests/hw3/test_ndarray.py::test_reduce_max[params1-cpu] PASSED           [ 75%]
tests/hw3/test_ndarray.py::test_reduce_max[params2-cpu] PASSED           [ 87%]
tests/hw3/test_ndarray.py::test_reduce_max[params3-cpu] PASSED           [100%]

=============

In [48]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cpu_reductions"

## Part 5: CPU Backend - Matrix multiplication

Implement the following functions in `ndarray_backend_cpu.cc`:

* `Matmul()`
* `MatmulTiled()`
* `AlignedDot()`

The first implementation, `Matmul()` can use the naive three-nested-for-loops algorithm for matrix multiplication.  However, the `MatmulTiled()` performs the same matrix multiplication on memory laid out in tiled form, i.e., as a contiguous 4D array
```c++
float[M/TILE][N/TILE][TILE][TILE];
```
Note that the Python `__matmul__` code already does the conversion to tiled form when all sizes of the matrix multiplication are divisible by `TILE`, so your code just needs to implement the multiplication in this form.  In order to make the methods efficient, you will want to make use of (after you implement it), the `AlignedDot()` function, which will enable the compiler to efficiently make use of vector operations and proper caching.  The output matrix will also be in the tiled form above, and the Python backend will take care of the conversion to a normal 2D array.

Note that in order to get the most speedup possible from you tiled version, you may want to use the clang compiler with colab instead of gcc.  To do this, run the following command before building your code.

In [49]:
!python3 -m pytest -v -k "matmul and cpu"

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collected 138 items / 128 deselected / 10 selected                             

tests/hw3/test_ndarray.py::test_matmul[16-16-16-cpu] PASSED              [ 10%]
tests/hw3/test_ndarray.py::test_matmul[8-8-8-cpu] PASSED                 [ 20%]
tests/hw3/test_ndarray.py::test_matmul[1-2-3-cpu] PASSED                 [ 30%]
tests/hw3/test_ndarray.py::test_matmul[3-4-5-cpu] PASSED                 [ 40%]
tests/hw3/test_ndarray.py::test_matmul[5-4-3-cpu] PASSED                 [ 50%]
tests/hw3/test_ndarray.py::test_matmul[64-64-64-cpu] PASSED              [ 60%]
tests/hw3/test_ndarray.py::test_matmul[72-72-72-cpu] PASSED              [ 70%]
tests/hw3/test_ndarray.py::test_matmul[72-73-74-cpu] PASSED              [ 80%]
tests/hw3/test

In [50]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cpu_matmul"

In [51]:
!export CXX=/usr/bin/clang++ && make

-- Found pybind11: /home/haozhe/anaconda3/lib/python3.11/site-packages/pybind11/include (found version "2.11.1")
-- Found cuda, building cuda backend
Wed Mar  6 13:10:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.01             Driver Version: 537.70       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro P600                    On  | 00000000:65:00.0  On |                  N/A |
| 34%   46C    P0              N/A /  N/A |   1310MiB /  2048MiB |      2%      Default |
|       

## Part 6: CUDA Backend - Compact and setitem

Implement the following functions in `ndarray_backend_cuda.cu`:
* `Compact()`
* `EwiseSetitem()`
* `ScalarSetitem()`


For this portion, you'll implement the compact and setitem calls in the CUDA backend.  This is fairly similar to the C++ version, however, depending on how you implemented that function, there could also be some substantial differences.  We specifically want to highlight a few differences between the C++ and the CUDA implementations, however.

First, as with the example functions implemented in the CUDA backend code, for all the functions above you will actually want to implement two functions: the basic functions listed above that you will call from Python, and the corresponding CUDA kernels that will actually perform the computation.  For the most part, we only provide the prototype for the "base" function in the `ndarray_backend_cuda.cu` file, and you will need to define and implement the kernel function yourself.  However, to see how these work, for the `Compact()` call we are providing you with the _complete_ `Compact()` call, and the function prototype for the `CompactKernel()` call.

One thing you may notice is the seemingly odd use of a `CudaVec` struct, which is a struct used to pass shape/stride parameters.  In the C++ version we used the STL `std::vector` variables to store these inputs (and the same is done in the base `Compact()` call, but CUDA kernels cannot operation on STL vectors, so something else is needed).  Furthermore, although we _could_ convert the vectors to normal CUDA arrays, this would be rather cumbersome, as we would need to call `cudaMalloc()`, pass the parameters as integer pointers, then free them after the calls.  Of course such memory management is needed for the actual underlying data in the array, but it seems like overkill to do it for just passing a variable-sized small tuple of shape/stride values.  The solution is to create a struct that has a "maximize" size for the number of dimensions an array can have, and then just store the actual shape/stride data in the first entries of these fields.  This is all done by the included `CudaVec` struct and `VecToCuda()` function, and you can just use these as provided for all the CUDA kernels that require passing shape/strides to the kernel itself.

The other (more conceptual) big difference between the C++ and CUDA implementations of `Compact()` is that in C++ you will typically loop over the elements of the non-compact array sequentially, which allows you to perform some optimizations with respect to computing the corresponding indices between the compact and non-compact arrays.  In CUDA, you cannot do this, and will need to implement code that can directly map from an index in the compact array to one in the strided array.

As before, we recommend you implement your code in such as way that it can easily be re-used between the `Compact()`, and `Setitem()` calls.  As a short note, remember that if you want to call a (separate, non-kernel) function from kernel code, you need to define it as a `__device__` function.


In [52]:
!python3 -m pytest -v -k "(compact or setitem) and cuda"

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collected 138 items / 123 deselected / 15 selected                             

tests/hw3/test_ndarray.py::test_compact[cuda-transpose] PASSED           [  6%]
tests/hw3/test_ndarray.py::test_compact[cuda-broadcast_to] PASSED        [ 13%]
tests/hw3/test_ndarray.py::test_compact[cuda-reshape1] PASSED            [ 20%]
tests/hw3/test_ndarray.py::test_compact[cuda-reshape2] PASSED            [ 26%]
tests/hw3/test_ndarray.py::test_compact[cuda-reshape3] PASSED            [ 33%]
tests/hw3/test_ndarray.py::test_compact[cuda-getitem1] PASSED            [ 40%]
tests/hw3/test_ndarray.py::test_compact[cuda-getitem2] PASSED            [ 46%]
tests/hw3/test_ndarray.py::test_compact[cuda-transposegetitem] PASSED    [ 53%]
tests/hw3/test

In [53]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cuda_compact_setitem"

## Part 7: CUDA Backend - Elementwise and scalar operations

Implement the following functions in `ndarray_backend_cuda.cu`:

* `EwiseMul()`, `ScalarMul()`
* `EwiseDiv()`, `ScalarDiv()`
* `ScalarPower()`
* `EwiseMaximum()`, `ScalarMaximum()`
* `EwiseEq()`, `ScalarEq()`
* `EwiseGe()`, `ScalarGe()`
* `EwiseLog()`
* `EwiseExp()`
* `EwiseTanh()`

Again, we don't provide these function prototypes, and you're welcome to use C++ templates or macros to make this implementation more compact.  You will also want to uncomment the appropriate regions of the Pybind11 code once you've implemented each function.

In [54]:
!python3 -m pytest -v -k "(ewise_fn or ewise_max or log or exp or tanh or (scalar and not setitem)) and cuda"

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collected 138 items / 115 deselected / 23 selected                             

tests/hw3/test_ndarray.py::test_ewise_fn[cuda-shape0-multiply] PASSED    [  4%]
tests/hw3/test_ndarray.py::test_ewise_fn[cuda-shape0-divide] PASSED      [  8%]
tests/hw3/test_ndarray.py::test_ewise_fn[cuda-shape0-add] PASSED         [ 13%]
tests/hw3/test_ndarray.py::test_ewise_fn[cuda-shape0-subtract] PASSED    [ 17%]
tests/hw3/test_ndarray.py::test_ewise_fn[cuda-shape0-equal] PASSED       [ 21%]
tests/hw3/test_ndarray.py::test_ewise_fn[cuda-shape0-greater_than] PASSED [ 26%]
tests/hw3/test_ndarray.py::test_ewise_fn[cuda-shape1-multiply] PASSED    [ 30%]
tests/hw3/test_ndarray.py::test_ewise_fn[cuda-shape1-divide] PASSED      [ 34%]
tests/hw3/tes

In [55]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cuda_ops"

## Part 8: CUDA Backend - Reductions


Implement the following functions in `ndarray_backend_cuda.cu`:

* `ReduceMax()`
* `ReduceSum()`

You can take a fairly simplistic approach here, and just use a separate CUDA thread for each individual reduction item: i.e., if there is a 100 x 20 array you are reducing over the second dimension, you could have 100 threads, each of which individually processed its own 20-dimensional array..  This is particularly inefficient for the `.max(axis=None)` calls, but we won't worry about this for the time being.  If you want a more industrial-grade implementation, you use a hierarchical mechanism that first aggregated across some smaller span, then had a secondary function that aggregated across _these_ reduced arrays, etc.  But this is not needed to pass the tests.

In [56]:
!python3 -m pytest -v -k "reduce and cuda"

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collected 138 items / 130 deselected / 8 selected                              

tests/hw3/test_ndarray.py::test_reduce_sum[params0-cuda] PASSED          [ 12%]
tests/hw3/test_ndarray.py::test_reduce_sum[params1-cuda] PASSED          [ 25%]
tests/hw3/test_ndarray.py::test_reduce_sum[params2-cuda] PASSED          [ 37%]
tests/hw3/test_ndarray.py::test_reduce_sum[params3-cuda] PASSED          [ 50%]
tests/hw3/test_ndarray.py::test_reduce_max[params0-cuda] PASSED          [ 62%]
tests/hw3/test_ndarray.py::test_reduce_max[params1-cuda] PASSED          [ 75%]
tests/hw3/test_ndarray.py::test_reduce_max[params2-cuda] PASSED          [ 87%]
tests/hw3/test_ndarray.py::test_reduce_max[params3-cuda] PASSED          [100%]

=============

In [57]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cuda_reductions"

## Part 9: CUDA Backend - Matrix multiplication

Implement the following functions in `ndarray_backend_cuda.cu`:

* `Matmul()`

Finally, as your final exercise, you'll implement matrix multiplication on the GPU.  Your implementation here can roughly follow the presentation in class.  While you can pass the tests using fairly naive code here (i.e., you could just have a separate thread for each (i,j) location in the matrix, doing the matrix multiplication efficiently (to make it actually faster than a CPU version) requires cooperative fetching and the block shared memory register tiling covered in class.  Try to implement using these methods, and see how much faster you can get your code than the C++ (or numpy) backends.


In [58]:
!python3 -m pytest -v -k "matmul and cuda"

============================= test session starts ==============================
platform linux -- Python 3.11.5, pytest-7.4.0, pluggy-1.0.0 -- /home/haozhe/anaconda3/bin/python3
cachedir: .pytest_cache
rootdir: /home/haozhe/DLsys/homework/hw3
plugins: anyio-3.5.0
collected 138 items / 128 deselected / 10 selected                             

tests/hw3/test_ndarray.py::test_matmul[16-16-16-cuda] PASSED             [ 10%]
tests/hw3/test_ndarray.py::test_matmul[8-8-8-cuda] PASSED                [ 20%]
tests/hw3/test_ndarray.py::test_matmul[1-2-3-cuda] PASSED                [ 30%]
tests/hw3/test_ndarray.py::test_matmul[3-4-5-cuda] PASSED                [ 40%]
tests/hw3/test_ndarray.py::test_matmul[5-4-3-cuda] PASSED                [ 50%]
tests/hw3/test_ndarray.py::test_matmul[64-64-64-cuda] PASSED             [ 60%]
tests/hw3/test_ndarray.py::test_matmul[72-72-72-cuda] PASSED             [ 70%]
tests/hw3/test_ndarray.py::test_matmul[72-73-74-cuda] PASSED             [ 80%]
tests/hw3/test

In [59]:
# !python3 -m mugrade submit "YOUR KEY HERE" -k "ndarray_cuda_matmul"